<a href="https://colab.research.google.com/github/Tkag0001/AI_and_Machine_Learning_for_Coders/blob/main/Chapter_7_Recurrent_Neural_Networks_for_Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chapter 7: Recurrent Neural Networks for Natural Language Processing

Chào mọi người nha, tụi mình lại gặp nhau rồi 🌻.
Ở chương này tụi mình sẽ cùng tìm hiểu về việc sử dụng các mạng hồi quy để xử lý các bài toán về ngôn ngữ tự nhiên nha.



Trước khi bước vào chương mới, tụi mình cùng ôn lại 2 chương trước đó nha.

Ở **Chương 5** mọi người đã được tìm hiểu về việc mã hóa các câu thành dạng chuỗi hay tensor của số thông qua cơ chế **Tokenizer**.

Ở **Chương 6** chúng ta lại tìm hiểu thêm về **vector biểu diễn** hay **cơ chế embedding**, thông qua việc đưa các từ lên không gian cao hơn giúp truy xuất được nhiều ngữ nghĩa hơn, đồng thời xây dựng mối liên kết giữa các vectors đại diện từ có nhóm ngữ nghĩa tương đương nhau.

Tuy nhiên bạn cũng nhận thấy rằng về cơ bản thì các phương pháp trên đều hoạt động dựa giống như một túi chứa các từ thôi, không phân biệt về thứ tự của chữ. Mà trong ngôn ngữ, **thứ tự** của các từ cũng đóng vai trò vô cùng **quan trọng về mặt ngữ nghĩa**.

Ví dụ: Từ **"blue"** để yên cho nó có vẻ khá thiếu nghĩa, cũng giống như từ "sky" vậy. Nhưng khi thêm một danh từ sau chúng sẽ lại ăn khớp với nhau đến kì lạ, giờ đây từ **"blue sky"** sẽ có nghĩa là **"bầu trời xanh"**. Ngoài ra còn có các từ khác như **"writing desk", "rain cloud"**

Các mô hình thông thường, chúng chỉ biết chúng có những từ đó và đưa lên không gian chiều cao hơn rồi đến các lớp tuyến tính và tính toán thôi nên khong mang tính ý nghĩa.



Chính vì mặt hạn chế đó nên chúng ta cần một hướng tiếp cận mới. Và đó là mô hình hồi quy sử dụng tròn mạng thần kinh neural.